In [1]:
// 📦 NuGet Package Reference: Microsoft Extensions AI
// This provides the core AI abstraction layer for .NET applications
#r "nuget: Microsoft.Extensions.AI, 9.9.0"

Installed Packages Microsoft.Extensions.AI, 9.9.0

In [2]:
// 🔗 Local Reference: OpenAI Agent Extensions
// This references the local build of the Microsoft Extensions AI Agents OpenAI library
// Contains OpenAI-specific implementations for the agent framework
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.AI.OpenAI\bin\Debug\net9.0\Microsoft.Agents.AI.OpenAI.dll"

In [3]:
// 🤖 Local Reference: Core Agent Framework
// This references the core Microsoft Extensions AI Agents library
// Provides the fundamental agent abstractions and base classes
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.AI\bin\Debug\net9.0\Microsoft.Agents.AI.dll"

In [4]:
// 🔌 NuGet Package Reference: OpenAI Integration
// Preview version of Microsoft Extensions AI OpenAI integration
// Provides OpenAI client and service implementations
#r "nuget: Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4"

Installed Packages Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4

In [5]:
// 🔧 NuGet Package Reference: Environment Variables
// DotNetEnv allows loading environment variables from .env files
// Essential for managing API keys and configuration securely
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages DotNetEnv, 3.1.1

In [6]:
// 📚 Import Required Namespaces
using System;                          // Core system functionality
using System.ComponentModel;           // For Description attributes on tool functions
using System.ClientModel;             // Client model abstractions

using Microsoft.Extensions.AI;        // Core AI abstractions and interfaces
using Microsoft.Agents.AI; // Agent framework classes and interfaces
using OpenAI;                         // OpenAI client library

In [7]:
// 🔧 Import Environment Variable Loading
// DotNetEnv namespace for loading .env files
using DotNetEnv;

In [8]:
// 🔧 Load Environment Variables from .env File
// This loads configuration from a .env file located 3 directories up
// Required variables: GITHUB_ENDPOINT, GITHUB_TOKEN, GITHUB_MODEL_ID
Env.Load("../../../.env");

In [9]:
// 🎲 Tool Function: Random Destination Generator
// This static method will be available to the agent as a callable tool
// The [Description] attribute helps the AI understand when to use this function
[Description("Provides a random vacation destination.")]
static string GetRandomDestination()
{
    // List of popular vacation destinations around the world
    var destinations = new List<string>
    {
        "Paris, France",
        "Tokyo, Japan", 
        "New York City, USA",
        "Sydney, Australia",
        "Rome, Italy",
        "Barcelona, Spain",
        "Cape Town, South Africa",
        "Rio de Janeiro, Brazil",
        "Bangkok, Thailand",
        "Vancouver, Canada"
    };

    // Generate random index and return selected destination
    var random = new Random();
    int index = random.Next(destinations.Count);
    return destinations[index];
}


In [10]:
var github_endpoint = Environment.GetEnvironmentVariable("GITHUB_ENDPOINT") ?? throw new InvalidOperationException("GITHUB_ENDPOINT is not set.");
var github_model_id = Environment.GetEnvironmentVariable("GITHUB_MODEL_ID") ?? "gpt-4o-mini";
var github_token = Environment.GetEnvironmentVariable("GITHUB_TOKEN") ?? throw new InvalidOperationException("GITHUB_TOKEN is not set.");

In [11]:
var openAIOptions = new OpenAIClientOptions()
{
    Endpoint= new Uri(github_endpoint)
};

In [12]:

var openAIClient = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions);

In [13]:
AIAgent agent = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions).GetChatClient(github_model_id).CreateAIAgent(
    instructions:"You are a helpful AI Agent that can help plan vacations for customers at random destinations", tools: [AIFunctionFactory.Create((Func<string>)GetRandomDestination)]);

In [14]:
Console.WriteLine(await agent.RunAsync("Plan me a day trip"));

Here's a delightful day trip plan for you in Barcelona, Spain:

### Morning
- **Breakfast at a Local Café**: Start your day with a traditional Spanish breakfast. Try "pa amb tomàquet" (bread with tomato) and a cup of coffee at a local café.

- **Visit La Sagrada Família**: Explore Gaudí's magnificent basilica. Make sure to book your tickets in advance to avoid long queues.

### Midday
- **Stroll through Park Güell**: Head to this colorful park designed by Gaudí. Enjoy the vibrant mosaics and stunning views of the city.

- **Lunch at a Tapas Bar**: Enjoy some delicious tapas at a local restaurant. Don’t miss trying “patatas bravas” and “croquetas”.

### Afternoon
- **Explore the Gothic Quarter**: Wander through the narrow streets filled with history, stunning architecture, and charming shops. Visit the Barcelona Cathedral.

- **Visit the Picasso Museum**: Discover a great collection of Picasso's works and learn about his early years in Barcelona.

### Evening
- **Relax at Barceloneta Be

In [15]:
await foreach (var update in agent.RunStreamingAsync("Plan me a day trip"))
{
    Console.Write(update);
}

Here's a plan for a day trip to Vancouver, Canada!

### Morning
- **Breakfast:** Start your day with breakfast at **Jam Cafe**, known for its delicious brunch menu and cozy atmosphere.
- **Stanley Park:** After breakfast, head to **Stanley Park**. You can rent a bike and cycle around the scenic Seawall, or take a leisurely stroll through the park's beautiful gardens.

### Midday
- **Granville Island:** Next, make your way to **Granville Island**. Explore the Public Market and sample local foods, browse artisan shops, and enjoy the vibrant atmosphere.
- **Lunch:** Grab lunch at **Granville Island Brewing**, where you can enjoy local craft beer and a tasty meal.

### Afternoon
- **Vancouver Aquarium:** Visit the **Vancouver Aquarium** located within Stanley Park. It's a fantastic place to learn about marine life and see various aquatic animals.
- **Gastown:** After the aquarium, stroll over to **Gastown**, Vancouver's oldest neighborhood, famous for its cobblestone streets and the iconic